# INFERNO: Inference-Aware Neural Optimisation

This notebook will demonstrate the effectiveness of the
*INFERNO: Inference-Aware Neural Optimisation* technique (as described in
[arxiv:1806.04743](https://arxiv.org/abs/1806.04743)) on a synthetic 3D inference problem
where the effect of nuisance parameters is relevant.



## Introduction

The staple of statistical infence is the is the probability density function
$p(\boldsymbol{x} | \boldsymbol{\theta})$,  where $\boldsymbol{x}$ are
the observed data and $\boldsymbol{\theta}$ are the parameters. In order to carry obtain inference about a set of i.i.d. observations
$D = \{\boldsymbol{x}_0, ..., \boldsymbol{x}_n\}$,
the density
$p(\boldsymbol{x} | \boldsymbol{\theta})$ plays a fundamental role linking the
observations with the model parameters, both in a classical or a Bayesian setting, because it serves as the basis to construct a likelihood function.

However, in many scientific disciplines such as population genetics, epidemiology or
experimental particle physics, the only way to realistically model observations and their
relation to the model parameters is by means of a complex simulation program.
In these cases, the link between the model parameters is only implicitly defined by a
generative procedure and statistical inference becomes more challenging.

Several approaches such as Approximate Bayesian Computation (ABC) or using
simplified synthetic likelihoods instead of the generative likelihood have been developed
to tackle the statistical inference problem when only forward simulations are available, in what are referred to as **Likelihood Free Inference** (LFI) techniques (see
paper for detailed bibliographic references).

If the dimensionality of the data $\boldsymbol{x}$ is high or the number of
observations $n$ is large, 
statistical inference using likelihood-free techniques can be computationally
intractable due to the curse of dimensionality. In these cases, we have to resort to based
the inference procedures on a summary statistic of the set of observations
$\boldsymbol{t}(D)$ instead of the original space. The choice of summary statistic
is critical and problem-dependent, a naive choice would most likely lead to the loss
of information about the parameters and thus a degradation on the resulting statistical
inference problem.

In this notebook, an approach to learn summary statistics using technique referred as Inference-Aware Neural Optimisation (referred to as INFERNO from now on) is presented and compared with alternatives using a practical benchmark example.

## Environment Setup

The implementation used in this notebook is heavily based in TensorFlow
and TensorFlow Probability, and the code used is freely available in
[this GitHub repository](https://github.com/pablodecm/paper-inferno/tree/improve_notebooks_readme).

It can either be used locally by installing the environment as described in the repository
readme or by using the binder or Google Colab services.
If you are seeing this in Google Colab please execute the following cell to install
the missing dependencies.

In [ ]:
# only run if executing in Google Colab
import os
os.environ["IS_COLAB"] = "True"
!git clone --branch improve_notebooks_readme https://github.com/pablodecm/paper-inferno.git
!pip install tensorflow-probability==0.6.0
!pip install corner
!pip install git+https://github.com/pablodecm/neyman.git@update_deps
# update reposity in case it has been updated
!cd paper-inferno; git pull
# go to folder as if executed locally
!cd paper-inferno/notebooks

Once the dependendencies have been installed, we can proceed to import of the modules that will be used in this notebook.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
# use default matplotlib style
plt.style.use('default')

from neyman.inferences import batch_hessian
import itertools as it
from tqdm import tnrange
import pandas as pd
import corner

if "IS_COLAB" in os.environ:
  repo_path = "/content/paper-inferno"
else:
  repo_path = ".."

import sys
sys.path.append(f"{repo_path}/code/")

from synthetic_3D_example import SyntheticThreeDimExample
from template_model import TemplateModel
import tensorflow_probability as tfp

from summary_statistic_computer import SummaryStatisticComputer

ds = tfp.distributions
ge = tf.contrib.graph_editor
k = tf.keras

font = {'size'   : 14}

matplotlib.rc('font', **font)
figure = {'figsize'   : (12,6),
          'max_open_warning': False}
matplotlib.rc('figure', **figure)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.logging.set_verbosity(tf.logging.ERROR)

# this dataframe will be used to keep the final results
columns = ["sigma_s (no nuisances)", "sigma_s (w/ nuisances)"]
unc_results = pd.DataFrame(columns=columns)

## Benchmark Description

The considered distributions is a mixture of two 3D distributions, referred as
$f_s(\textbf{x})$ and $f_b(\textbf{x})$. 

The background distribution $f_s(\textbf{x})$ consist of a 2D gaussian in the
fist two dimensions centred at $(r,0.)$ where r is a nuisance parameter loosely constrained
around $r=2.$. The last dimension is a exponential distribution with a rate $\lambda$ loosely
constrained around $\lambda=3$ which will be the second nuisance parameter.

The  signal distribution $f_s(\textbf{x})$ consist of a 2D gaussian in the
fist two dimensions centred at $(0.,0.)$.
The last dimension is a exponential distribution with a rate of $2.0$.

Analytically the mixture density considered can be expressed as:
$$ p(\textbf{x}) = \mu \cdot s(\textbf{x}) + (1-\mu) \cdot b(\textbf{x}) $$
However, a different parametetrization will be considered, in which our parameter of interest
will be denoted as $s_{\textrm{exp}}$ corresponding to the expected number of signal observations, which we expect to be around 100.
The expected number of background observations will be the third nuisance parameter, which will
be loosely constrained around $b_{\textrm{exp}} = 1000$. The total number of
expected observations will be distributed as $\textrm{Poisson}(s_{\textrm{exp}}+b_{\textrm{exp}}$).
The mixture fraction $\mu$ can be expressed in this parametrization as:
$$ \mu = \frac{s_{\textrm{exp}}}{s_{\textrm{exp}}+b_{\textrm{exp}}}$$

In [ ]:

def show_function_code(function):
  
    import inspect
    from pygments import highlight
    from pygments.lexers import PythonLexer
    from pygments.formatters import HtmlFormatter
    from IPython.core.display import HTML
    
    source_code = "".join(inspect.getsourcelines(function)[0])

    return HTML(highlight(source_code, PythonLexer(), HtmlFormatter(full=True)))
  

show_function_code(SyntheticThreeDimExample.__init__)

In [ ]:
show_function_code(SyntheticThreeDimExample.transform_bkg)

In [ ]:
problem = SyntheticThreeDimExample()

x_values = tf.placeholder(dtype=tf.float32, shape=(None, 3), name="x_values")

phs = { problem.r_dist : 2.,
        problem.b_rate : 3.,
        problem.b_exp : 1000,
        problem.s_exp : 50}

In [ ]:

n_s = 50000

s_sam = problem.s_dist.sample(n_s, seed=27)
b_sam = problem.b_dist.sample(n_s, seed=27)
with tf.Session() as sess:
  b_sample = sess.run(b_sam, phs)
  s_sample = sess.run(s_sam, phs)

labels = [r"$x_0$", "$x_1$", "$x_2$"]
ran = [(-10,10),
       (-10,10),
       (0,4)]
bins = [np.linspace(-10,10,40, endpoint=True),
        np.linspace(-10,10,40, endpoint=True),
        np.linspace(0,4,20, endpoint=True)]

# 2D levels to use
levels = 1.0 - np.exp(-0.5 * np.array([1.,2.,3.]) ** 2)

fig = corner.corner(b_sample, bins=bins,range=ran, color="blue",weights=np.ones(n_s),
                    smooth=0.95, labels=labels,levels=levels,plot_datapoints=False)
fig = corner.corner(s_sample, bins=bins,range=ran, color="red",weights=np.ones(n_s),
                    smooth=0.95, labels=labels,levels=levels,plot_datapoints=False,
                    fig=fig)

# set 1D hist y-scales
scales = [12000, 12000, 25000]
n_dim = 3
axes = np.array(fig.axes).reshape((n_dim, n_dim))
for i in range(n_dim):
  ax = axes[i, i]
  ax.set_ylim([0,scales[i]])

#fig.savefig("../paper/gfx/figure2a.pdf",bbox_inches="tight")

In [ ]:

n_s = 50000

with tf.Session() as sess:
  weights=np.ones(s_sample.shape[0])
  m_sample = sess.run(problem.m_dist.sample(n_s, seed=17), phs)

labels = [r"$x_0$", "$x_1$", "$x_2$"]
ran = [(-10,10),
       (-10,10),
       (0,4)]
bins = [np.linspace(-10,10,40, endpoint=True),
        np.linspace(-10,10,40, endpoint=True),
        np.linspace(0,4,20, endpoint=True)]

# 2D levels to use
levels = 1.0 - np.exp(-0.5 * np.array([1.,2.,3.]) ** 2)

fig = corner.corner(m_sample, bins=bins, range=ran,
                    levels=levels,color="black",weights=np.ones(n_s),
                    smooth=0.9999, labels=labels,plot_datapoints=False)

# set 1D hist y-scales
scales = [12000, 12000, 25000]
n_dim = 3
axes = np.array(fig.axes).reshape((n_dim, n_dim))
for i in range(n_dim):
  ax = axes[i, i]
  ax.set_ylim([0,scales[i]])

#fig.savefig("../paper/gfx/figure2b.pdf",bbox_inches="tight")

In [ ]:
with tf.Session() as sess:
  train_arrays = sess.run(problem.train_data())
  valid_arrays = sess.run(problem.valid_data())   
  test_arrays =  sess.run(problem.test_data())   

## Classifier training

In [ ]:
from synthetic_3D_cross_entropy import SyntheticThreeDimCrossEntropy

clf_name = "NN Classifier"
seed = 17

n_epochs = 100
lr = 1e-3
batch_size = 32
xe_path = f"xe_lr_{lr}_n_e_{n_epochs}_seed_{seed}"

xe_model = SyntheticThreeDimCrossEntropy(model_path=xe_path,
                                         seed=seed)

xe_model.fit(n_epochs=n_epochs, lr=lr,batch_size=batch_size, seed=seed)

In [ ]:
ssc = SummaryStatisticComputer()

In [ ]:
clf_shapes = ssc.classifier_shapes(xe_path)

In [ ]:
from template_model import TemplateModel

def compute_fisher(shapes):
  with tf.Session() as sess:
    tm = TemplateModel()
    tm.templates_from_dict(shapes)
    fisher = tm.asimov_hess()
  return fisher

In [ ]:
clf_fish = compute_fisher(clf_shapes)
sigma_s_no_nuis = clf_fish.marginals(["s_exp"])["s_exp"]
sigma_s_w_nuis = clf_fish.marginals(["s_exp","r_dist","b_rate"])["s_exp"]
unc_results.loc[clf_name] = [sigma_s_no_nuis, sigma_s_w_nuis]
unc_results

In [ ]:

s_exp_scan = np.linspace(20.,80.,61, endpoint=True)
pars = ["r_dist","b_rate"]
n_steps = 5
d = 0
step_size = 0.1

def profile_likelihood(shapes, pars = ["r_dist","b_rate"]):
  
  with tf.Session() as sess:
    tm = TemplateModel(multiple_pars=True)
    tm.templates_from_dict(shapes)
    par_phs = {tm.r_dist : 2.0*np.ones_like(s_exp_scan),
             tm.b_rate : 3.0*np.ones_like(s_exp_scan),
             tm.s_exp :  s_exp_scan,
             tm.b_exp : 1000.0*np.ones_like(s_exp_scan)} 
   
  
    asimov_data = tm.asimov_data(sess=sess)
  
    obs_phs = {tm.obs : asimov_data}
    mod_phs = par_phs.copy()
    # get likelihood before changing pars
    nll, sub_hess, sub_grad = tm.hessian_and_gradient(pars=pars,
                                                      par_phs=mod_phs, obs_phs=obs_phs)
      
    # profile likelihood with Newton method
    for i in range(n_steps):
      newton_step =  np.matmul(np.linalg.inv(sub_hess+d*np.eye(len(pars))),sub_grad[:,:,np.newaxis])
      mod_phs[tm.r_dist] = mod_phs[tm.r_dist] - step_size*newton_step[:,0,0]
      mod_phs[tm.b_rate] = mod_phs[tm.b_rate] - step_size*newton_step[:,1,0]
      p_nll, sub_hess, sub_grad = tm.hessian_and_gradient(pars=["r_dist","b_rate"],
                                                          par_phs=mod_phs, obs_phs=obs_phs)
      
    return nll, p_nll


In [ ]:
nll, p_nll = profile_likelihood(clf_shapes)

fig, ax = plt.subplots()
ax.plot(s_exp_scan, nll, label="no nuisances" )
ax.plot(s_exp_scan, p_nll, label="w/ nuisances")
ax.legend()

In [ ]:
def shape_variation_plot(shape_dict, bins=None):
  fig, ax = plt.subplots(2,1, figsize=(8,8),sharex=True)
  fig.subplots_adjust(hspace = 0.05)
  
  
  if bins is None: 
    bins = np.linspace(0,1,11,endpoint=True)
  centers =   (bins[1:]+bins[:-1])/2.
  width = (bins[1:]-bins[:-1])
  
  plot_options = {("sig",) : (r"signal","red"),
                 ("bkg",2.,3.) : (r"background","blue")}
                 
  for k, option in plot_options.items():
    clf_shapes_norm = shape_dict[k]/shape_dict[k].sum()
    ax[0].bar(x=centers, height=clf_shapes_norm,
              width=width,color=option[1], alpha=0.35,
              label=option[0])
  
  exp_bkg =  (shape_dict[("bkg",2.,3.)]/shape_dict[("bkg",2.,3.)].sum())*1000.
  exp_sig = (shape_dict[("sig",)]/shape_dict[("sig",)].sum())*50.
  sig_art = ax[1].bar(x=centers, height=exp_sig/exp_bkg,width=width,
                      color="red", label="signal", alpha=0.35)
  
  plot_options = {("bkg",2.0,2.5) : (r"v","green"),
                  ("bkg",2.0,3.5) : (r"^","green"),
                  ("bkg",1.8,3.0) : (r"v","purple"),
                  ("bkg",2.2,3.0) : (r"^","purple")}
  
  var_lines = []
  var_labels = []
  for k, option in plot_options.items():
    exp_shift_bkg = (shape_dict[k]/shape_dict[k].sum())*1000.
    label = f"bkg $r={k[1]}$ $\\lambda={k[2]}$"
    var_lines.append(ax[1].errorbar(x=centers, y=(exp_shift_bkg-exp_bkg)/exp_bkg,
                     xerr=width/2.,fmt=".", color=option[1],
                     alpha=0.55, marker=option[0]))
    var_labels.append(label)
  
  ax[0].legend(loc="upper center", frameon=False)
  ax[0].set_ylim(top=0.7)
  ax[0].set_ylabel("counts (normalised)")
  
  leg_0 = ax[1].legend(var_lines,var_labels,loc="lower center", ncol=2,frameon=False)
  leg_1 = ax[1].legend([sig_art],["signal"],loc="upper center", frameon=False)

  ax[1].set_ylim([-0.75,0.75])
  ax[1].set_xlim([bins.min(),bins.max()])
  
  ax[1].add_artist(leg_0)
  ax[1].set_ylabel("relative variation")
  ax[1].yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(0.2))
  
  fig.align_ylabels()

  
  return ax, fig

In [ ]:
axs, fig = shape_variation_plot(clf_shapes)
axs[1].set_xlabel("neural network classifier output")
#fig.savefig("../paper/gfx/figure3a.pdf",bbox_inches="tight")

## Analytical likelihood ratio

Probabilistic classifiers trained to differenciate signal and background observation can be used to approximate the likelihood ratio. In this
particular case, we can easily approximate the likelihood ratio analytically.

$$t_\frac{s}{s+b}(\textbf{x}) = \frac{f_s(\textbf{x})}{f_s(\textbf{x}) + f_b(\textbf{x})}$$

where $f_s$ is fully specified but $f_b$ depends on $r$ and $\lambda$, which for
a probabilistic classifier are typically taken as fixed at certain values.

In [ ]:
opt_name = "Optimal Classifier"

opt_shapes = ssc.optimal_shapes()
opt_fish = compute_fisher(opt_shapes)

sigma_s_no_nuis = opt_fish.marginals(["s_exp"])["s_exp"]
sigma_s_w_nuis = opt_fish.marginals(["s_exp","r_dist","b_rate"])["s_exp"]
unc_results.loc[opt_name] = [sigma_s_no_nuis, sigma_s_w_nuis]
unc_results

In [ ]:
axs, fig = shape_variation_plot(opt_shapes)

axs[1].set_xlabel("optimal classifier output")
#fig.savefig("../paper/gfx/figure3b.pdf",bbox_inches="tight")


## INFERNO training

In [ ]:
from synthetic_3D_inferno import SyntheticThreeDimInferno

inf_name = "INFERNO"

n_epochs = 200
lr = 1e-6
batch_size = 1000
t_train = 0.1
t_eval = 0.05

n_inits = 100
seed = 7


pars = ["s_exp", "r_dist", "b_rate"]


inf_path = f"inf_ne_{n_epochs}_lr_{lr}_bs_{batch_size}_t_{t_train}"

inferno = SyntheticThreeDimInferno(model_path=inf_path, poi="s_exp",
                                    pars=pars, seed=seed)
inferno.fit(n_epochs=n_epochs, lr=lr, batch_size=batch_size,
            temperature=t_train, seed=seed)

inf_fisher, inf_aux_fisher = inferno.eval_hessian(temperature=t_eval)

In [ ]:
loss_valid = np.array(inferno.history["loss_valid"])
plt.plot(loss_valid[:,0], loss_valid[:,1])

In [ ]:
inf_shapes = ssc.inferno_shapes(inf_path)
inf_fish = compute_fisher(inf_shapes)

In [ ]:
inf_fish.marginals(["s_exp"])["s_exp"]
sigma_s_no_nuis = inf_fish.marginals(["s_exp"])["s_exp"]
sigma_s_w_nuis = inf_fish.marginals(["s_exp","r_dist","b_rate"])["s_exp"]
unc_results.loc[inf_name] = [sigma_s_no_nuis, sigma_s_w_nuis]

unc_results

In [ ]:
nll, p_nll = profile_likelihood(inf_shapes)

fig, ax = plt.subplots()
ax.plot(s_exp_scan, nll, label="no nuisances" )
ax.plot(s_exp_scan, p_nll, label="w/ nuisances")
ax.legend()

In [ ]:
axs, fig = shape_variation_plot(inf_shapes)

axs[1].set_xlabel("inferno output")
#axs[0].set_ylim(top=0.01)

## Analytical Inference

In [ ]:
from extended_model import ExtendedModel

lik_name = "Analytical Inference"
problem = SyntheticThreeDimExample()


In [ ]:
x_values = tf.placeholder(dtype=tf.float32, shape=(None, 3), name="x_values")

em = ExtendedModel(problem)

In [ ]:
bkg_t = problem.transform_bkg(x_values)

with tf.Session() as sess:
  valid_arrays = sess.run(problem.valid_data())   

In [ ]:
with tf.Session() as sess:
  bkg_t_arr = sess.run(bkg_t, {x_values : valid_arrays["bkg"]})
  obs_phs = {em.s_n_exp : 50.,
               em.b_n_exp : 1000.,
               em.s_data : valid_arrays["sig"],
               em.b_data : bkg_t_arr }
  e_hess, e_hess_aux = em.hess(par_phs={},obs_phs=obs_phs)   

In [ ]:
e_hess.marginals(["s_exp"])["s_exp"]

sigma_s_no_nuis = e_hess.marginals(["s_exp"])["s_exp"]
sigma_s_w_nuis = e_hess.marginals(["s_exp","r_dist","b_rate"])["s_exp"]
unc_results.loc[lik_name] = [sigma_s_no_nuis, sigma_s_w_nuis]

unc_results

## BYOS: Bring your Own Summary



Here the example of a hand-made summary.

In [ ]:
naive_name = "Naive Summary"

def naive_summary(data):
  return np.log(np.abs(data[:,1]))


# you can also choose the binning 
bins = np.linspace(0,2.5,11,endpoint=True)
naive_shapes = ssc.generic_shapes(naive_summary,bins)

axs, fig = shape_variation_plot(naive_shapes, bins)

In [ ]:
naive_fish = compute_fisher(naive_shapes)
# we do not need to consider the r_dist or b_rate parameters because only
# affects the first and third dimension respectively
# In fact it will make the inverse Hessian singular because this naive
# summary statistic does not provide information about those nuisance parameters

naive_fish.marginals(["s_exp"])["s_exp"]
sigma_s_w_nuis = naive_fish.marginals(["s_exp"])["s_exp"]
unc_results.loc[naive_name] = [sigma_s_no_nuis, sigma_s_w_nuis]

unc_results

Alternatively we could use machine learning model trained with other library or method, here I provide a working PyTorch based binary classifier as a skeleton example.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as utils

class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.fc1 = nn.Linear(3,50)
    self.fc2 = nn.Linear(50,100)
    self.fc3 = nn.Linear(100,100)
    self.out = nn.Linear(100,1)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    return torch.sigmoid(self.out(x))


model = Model()
opt = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.BCELoss()
 
X = np.vstack([train_arrays["sig"],train_arrays["bkg"]])
y = np.hstack([np.ones(train_arrays["sig"].shape[0], dtype=np.float32),
               np.zeros(train_arrays["bkg"].shape[0], dtype=np.float32)])[:,np.newaxis]

permutation = np.random.permutation(X.shape[0])
X = torch.as_tensor(X[permutation])
y = torch.as_tensor(y[permutation])

trainset = utils.TensorDataset(X,y)
trainloader = utils.DataLoader(trainset, batch_size=64, shuffle=True)

loss_history = []
loss_batch_number = []
batch_number = 0
for epoch in range(5):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, target = data

        # zero the parameter gradients
        opt.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        opt.step()

        # print statistics
        running_loss += loss.item()
        batch_number += 1
        if i % 100 == 99:    # print every 100 mini-batches
            loss_history.append(running_loss/100.)
            loss_batch_number.append(batch_number)
            running_loss = 0.0

plt.plot(loss_batch_number,loss_history);

In [ ]:
byos_name = "BYOS Summary"

def byos_summary(data):
  # you can use anything that transforms each observation to a one dimensional
  # summary (e.g. a trained model neural network model output using your
  # preferred library)
  # both data and transformed_data are numpy arrays
  
  # could also be loaded from a file
  transformed_data = model(torch.as_tensor(data)).data
  return transformed_data


# you can also choose the binning so it is adecuate for your summary
bins = np.linspace(0,1,11,endpoint=True)

# compute shapes and variation
byos_shapes = ssc.generic_shapes(byos_summary,bins)
axs, fig = shape_variation_plot(byos_shapes, bins)

In [ ]:
byos_fish = compute_fisher(byos_shapes)

sigma_s_no_nuis = byos_fish.marginals(["s_exp"])["s_exp"]
sigma_s_w_nuis = byos_fish.marginals(["s_exp","r_dist","b_rate"])["s_exp"]
unc_results.loc[byos_name] = [sigma_s_no_nuis, sigma_s_w_nuis]

unc_results

In [ ]:
nll, p_nll = profile_likelihood(byos_shapes)

fig, ax = plt.subplots()
ax.plot(s_exp_scan, nll, label="no nuisances" )
ax.plot(s_exp_scan, p_nll, label="w/ nuisances")
ax.legend()